# Rescue of Legacy Goodreads Data
Goodreads API has been depreciated for [several years](https://help.goodreads.com/s/article/Does-Goodreads-support-the-use-of-APIs)  
Exporting my data to store it elsewhere.

In [74]:
import pandas as pd

In [75]:
goodreads = pd.read_csv('goodreads_library_export.csv').sort_values(by=['Date Added'])

In [76]:
goodreads

,Book Id,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,...,Date Read,Date Added,Bookshelves,Bookshelves with positions,Exclusive Shelf,My Review,Spoiler,Private Notes,Read Count,Owned Copies
43,7745,Fear and Loathing in Las Vegas,Hunter S. Thompson,"Thompson, Hunter S.",Ralph Steadman,"=""0679785892""","=""9780679785897""",5,4.07,Vintage Books,...,2014/08/26,2014/08/26,NaN,NaN,read,NaN,NaN,NaN,1,0
44,4671,The Great Gatsby,F. Scott Fitzgerald,"Fitzgerald, F. Scott",NaN,"=""""","=""""",4,3.93,Scribner,...,2014/08/26,2014/08/26,NaN,NaN,read,NaN,NaN,NaN,1,0
64,629,Zen and the Art of Motorcycle Maintenance: An ...,Robert M. Pirsig,"Pirsig, Robert M.",NaN,"=""0060589469""","=""9780060589462""",5,3.78,HarperTorch,...,2016/08/07,2016/08/07,NaN,NaN,read,NaN,NaN,NaN,1,0
66,15815364,How to Get Filthy Rich in Rising Asia,Mohsin Hamid,"Hamid, Mohsin",NaN,"=""1594487294""","=""9781594487293""",5,3.81,Riverhead Books,...,2016/10/03,2016/09/30,NaN,NaN,read,NaN,NaN,NaN,1,0
65,2165,The Old Man and the Sea,Ernest Hemingway,"Hemingway, Ernest",NaN,"=""0684830493""","=""9780684830490""",5,3.80,Scribner,...,2017/02/22,2017/02/22,NaN,NaN,read,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,570933,Practical Common Lisp,Peter Seibel,"Seibel, Peter",NaN,"=""1590592395""","=""9781590592397""",5,4.15,Apress,...,NaN,2022/05/22,NaN,NaN,read,NaN,NaN,NaN,1,0
3,20873338,Clojure for the Brave and True,Daniel Higginbotham,"Higginbotham, Daniel",NaN,"=""""","=""""",4,4.27,No Starch Press,...,NaN,2022/05/22,NaN,NaN,read,NaN,NaN,NaN,1,0
2,108428,Are Prisons Obsolete?,Angela Y. Davis,"Davis, Angela Y.",NaN,"=""1583225811""","=""9781583225813""",5,4.54,Seven Stories Press,...,NaN,2022/05/22,NaN,NaN,read,NaN,NaN,NaN,1,0
1,23632478,Solaris,Stanisław Lem,"Lem, Stanisław",Bill Johnston,"=""8363471410""","=""""",5,3.99,Pro Auctore Wojciech Zemek,...,NaN,2023/01/11,NaN,NaN,read,NaN,NaN,NaN,1,0


In [77]:
nested = []
for index, row in goodreads.iterrows():
    # Start with the Markdown header for the table
    md_description = "### Legacy Goodreads Data\n"
    
    # Add each detail as a row in the Markdown table
    for col_name, value in row.items():
        if not pd.isnull(value):  # Skip the 'Title' column for the description
            md_description += f"- **{col_name}** : {value}\n"
    
    # Create the nested dictionary
    nested_dict = {
        'name': row['Title'],
        'desc': md_description
    }
    nested.append(nested_dict)

In [72]:
import os
from trello import TrelloClient
from dotenv import load_dotenv
import json

load_dotenv()

# Trello API credentials
API_KEY = os.environ['TRELLO_API_KEY']
API_SECRET = os.environ['TRELLO_OAUTH_SECRET']
TOKEN = os.environ['TRELLO_TOKEN']

# Trello board and list IDs
BOARD_NAME = 'Library'
LIST_NAME = 'Completed'

client = TrelloClient(
    api_key=API_KEY,
    api_secret=API_SECRET,
    token=TOKEN,
)

board = next(x for x in client.list_boards() if x.name == BOARD_NAME)
completed_list = next((lst for lst in board.list_lists() if lst.name == LIST_NAME), None)
if not completed_list: 
    print(f"List '{LIST_NAME}' not found in board")
    exit(1)

completed_books = [{'name': card.name, 'desc': card.description} for card in completed_list.list_cards()]

In [80]:
for book in nested[::-1]:
    print(f"adding {book['name']}")
    new_card = completed_list.add_card(
        name=book['name'],
        desc=book['desc'],
        position='bottom'
    )

adding The Order of Time
adding Solaris
adding Are Prisons Obsolete?
adding Clojure for the Brave and True
adding Practical Common Lisp
adding Digital Minimalism: Choosing a Focused Life in a Noisy World
adding The Divide: A Brief Guide to Global Inequality and its Solutions
adding Silencing the Past: Power and the Production of History
adding An Indigenous Peoples' History of the United States (ReVisioning American History, #3)
adding The Peregrine
adding Structure and Interpretation of Computer Programs
adding Homewaters: A Human and Natural History of Puget Sound
adding Adam Smith in Beijing: Lineages of the Twenty-First Century
adding Animal Liberation
adding JFK and the Unspeakable: Why He Died & Why It Matters
adding The Devil's Chessboard: Allen Dulles, the CIA, and the Rise of America's Secret Government
adding Imperialism: The Highest Stage of Capitalism
adding Inventing the Future (revised and updated edition): Postcapitalism and a World Without Work
adding Breakfast of Champ

In [86]:
import requests

def get_book_info_from_title(title, api_key):
    formatted_title = '+'.join(title.split())
    url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{formatted_title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()

        # Parse the response to find the ISBN and the canonical volume link
        items = data.get('items', [])
        if not items:
            print(f"No results found for title: {title}")
            return None, None
        else:
            # Assuming the first result is the desired book
            book = items[0]
            volume_info = book['volumeInfo']
            industry_identifiers = volume_info.get('industryIdentifiers', [])
            isbn_13 = None
            isbn_10 = None
            for identifier in industry_identifiers:
                if identifier['type'] == 'ISBN_13':
                    isbn_13 = identifier['identifier']
                elif identifier['type'] == 'ISBN_10':
                    isbn_10 = identifier['identifier']

            # Use the ISBN-13 if available, else fall back to ISBN-10
            isbn = isbn_13 if isbn_13 else isbn_10
            link_val = volume_info.get('infoLink', '')
            return isbn, link_val
    else:
        print(f"Failed to fetch data for title: {title}")
        return None, None

In [92]:
board = next(x for x in client.list_boards() if x.name == BOARD_NAME)
completed_list = next((lst for lst in board.list_lists() if lst.name == LIST_NAME), None)
if not completed_list: 
    print(f"List '{LIST_NAME}' not found in board")
    exit(1)

for card in completed_list.list_cards():
    isbn, google_books_link = get_book_info_from_title(card.name, api_key)

    # If an ISBN and link were found, update the card's description
    if isbn and google_books_link:
        new_description = f"**ISBN:** {isbn}\n**Google Books Link:** [View Book]({google_books_link})\n\n{card.description}"
        card.set_description(new_description)
        print(f"Updated card '{card.name}' with ISBN and link.")
    else:
        print(f"ISBN/Link not found for '{card.name}'. No updates made.")

Updated card 'The Deficit Myth' with ISBN and link.
Updated card 'The Hidden Life of Trees' with ISBN and link.
Updated card 'A Mind For Numbers' with ISBN and link.
